In [ ]:
import os

import matplotlib.pyplot as plt

from cedtrainscheduler.simulator.config import SimulatorConfig
from cedtrainscheduler.simulator.simulator import Simulator


def run_simulation(scheduler_name, fs_config_path):
    """运行模拟并返回结果"""
    current_dir = os.getcwd()  # 使用 os.getcwd() 获取当前工作目录
    base_dir = os.path.dirname(current_dir)


    config = SimulatorConfig(
        cluster_config_path=os.path.join(base_dir, "cedtrainscheduler/cases/cluster/cluster_config.json"),
        fs_config_path=os.path.join(base_dir, "cedtrainscheduler/cases/cluster/"+fs_config_path),
        scheduler_name=scheduler_name,
        task_config_path=os.path.join(base_dir, "cedtrainscheduler/cases/task/case_5000_tasks.csv"),
        output_path=os.path.join(base_dir, "examples/outputs"),
    )

    simulator = Simulator(config)
    results = simulator.simulation()
    return results

def plot_results(results_dict_light, results_dict_heavy):
    """绘制结果图"""
    import seaborn as sns
    from matplotlib.ticker import FuncFormatter

    # 使用 seaborn 风格
    sns.set(style="whitegrid")

    # 指标名称
    metrics_names = ['Queue Delay', 'JCT', 'Makespan']
    width = 0.15  # 调整条形图的宽度
    x_positions = [0, 1]  # 0 for light-data, 1 for heavy-data

    # 自定义颜色，为每个调度器分配不同的颜色
    colors = sns.color_palette("husl", len(results_dict_light))

    # 分别为每个指标创建一个图形窗口
    for metric_name in metrics_names:
        fig, ax = plt.subplots(figsize=(10, 6))

        for i, (scheduler_name, metrics_light) in enumerate(results_dict_light.items()):
            metrics_heavy = results_dict_heavy[scheduler_name]

            if metric_name == 'Queue Delay':
                value_light = metrics_light.avg_queue_time
                value_heavy = metrics_heavy.avg_queue_time
            elif metric_name == 'JCT':
                value_light = metrics_light.avg_running_time
                value_heavy = metrics_heavy.avg_running_time
            elif metric_name == 'Makespan':
                value_light = metrics_light.total_runtime
                value_heavy = metrics_heavy.total_runtime

            # 绘制轻量版和重版的条形图
            ax.bar(x_positions[0] + i * width - (len(results_dict_light) - 1) * width / 2, value_light, width, color=colors[i], label=scheduler_name)
            ax.bar(x_positions[1] + i * width - (len(results_dict_light) - 1) * width / 2, value_heavy, width, color=colors[i])

        ax.set_title(f'Comparison of Schedulers by {metric_name}', fontsize=16)
        ax.set_ylabel('Time (10^4)s', fontsize=14)
        ax.set_xticks(x_positions)
        ax.set_xticklabels(['Light Data', 'Heavy Data'], fontsize=12)
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y / 1e4:.1f}'))

        # 添加图例，显示所有调度器名称
        ax.legend(fontsize=12)

        # 调整布局
        plt.tight_layout()
        plt.show()

# 假设 results_dict_light 和 results_dict_heavy 是两个模拟器的结果字典
# 运行不同调度器的模拟
schedulers = ["k8s-data", "k8s", "sjf-data", "sjf", "ced"]
results_dict_light = {}
results_dict_heavy = {}

for scheduler in schedulers:
    # 使用轻量版配置运行模拟
    results_light = run_simulation(scheduler, fs_config_path='fs_config_light.json')
    results_dict_light[scheduler] = results_light
    print(results_light)

    # 使用重版配置运行模拟
    results_heavy = run_simulation(scheduler, fs_config_path='fs_config_heavy.json')
    results_dict_heavy[scheduler] = results_heavy
    print(results_heavy)

# 绘制结果
plot_results(results_dict_light, results_dict_heavy)

In [ ]:
# 计算 ced 的 JCT 降低百分比相对于其他调度器
def calculate_ced_jct_reduction(results_dict_light, results_dict_heavy):
    jct_reduction_light = {}
    jct_reduction_heavy = {}

    for scheduler in results_dict_light.keys():
        if scheduler == 'ced':
            continue

        jct_ced_light = results_dict_light['ced'].avg_running_time
        jct_ced_heavy = results_dict_heavy['ced'].avg_running_time

        jct_light = results_dict_light[scheduler].avg_running_time
        jct_heavy = results_dict_heavy[scheduler].avg_running_time

        # 计算 ced 相对于其他调度器的降低百分比
        reduction_light = ((jct_light - jct_ced_light) / jct_light) * 100
        reduction_heavy = ((jct_heavy - jct_ced_heavy) / jct_heavy) * 100

        jct_reduction_light[scheduler] = reduction_light
        jct_reduction_heavy[scheduler] = reduction_heavy

    return jct_reduction_light, jct_reduction_heavy

# 获取 ced 的 JCT 降低百分比
jct_reduction_light, jct_reduction_heavy = calculate_ced_jct_reduction(results_dict_light, results_dict_heavy)

# 输出结果
print("ced 的 JCT 降低百分比相对于其他调度器:")
for scheduler in jct_reduction_light.keys():
    print(f"{scheduler}: Light 情况下降低了 {jct_reduction_light[scheduler]:.2f}%, Heavy 情况下降低了 {jct_reduction_heavy[scheduler]:.2f}%")


# 计算 ced 的 Makespan 降低百分比相对于其他调度器
def calculate_ced_makespan_reduction(results_dict_light, results_dict_heavy):
    makespan_reduction_light = {}
    makespan_reduction_heavy = {}

    for scheduler in results_dict_light.keys():
        if scheduler == 'ced':
            continue

        makespan_ced_light = results_dict_light['ced'].total_runtime
        makespan_ced_heavy = results_dict_heavy['ced'].total_runtime

        makespan_light = results_dict_light[scheduler].total_runtime
        makespan_heavy = results_dict_heavy[scheduler].total_runtime

        # 计算 ced 相对于其他调度器的降低百分比
        reduction_light = ((makespan_light - makespan_ced_light) / makespan_light) * 100
        reduction_heavy = ((makespan_heavy - makespan_ced_heavy) / makespan_heavy) * 100

        makespan_reduction_light[scheduler] = reduction_light
        makespan_reduction_heavy[scheduler] = reduction_heavy

    return makespan_reduction_light, makespan_reduction_heavy

# 获取 ced 的 Makespan 降低百分比
makespan_reduction_light, makespan_reduction_heavy = calculate_ced_makespan_reduction(results_dict_light, results_dict_heavy)

# 输出结果
print("ced 的 Makespan 降低百分比相对于其他调度器:")
for scheduler in makespan_reduction_light.keys():
    print(f"{scheduler}: Light 情况下降低了 {makespan_reduction_light[scheduler]:.2f}%, Heavy 情况下降低了 {makespan_reduction_heavy[scheduler]:.2f}%")